In [1]:
!rm -rf /content/Moal
!git clone https://github.com/nqh-tq-32052503/Moal.git
%cd /content/Moal/Moal

Cloning into 'Moal'...
remote: Enumerating objects: 217, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 217 (delta 32), reused 32 (delta 25), pack-reused 171 (from 1)
Receiving objects: 100% (217/217), 464.31 KiB | 11.91 MiB/s, done.
Resolving deltas: 100% (105/105), done.
/content/Moal/Moal


In [2]:
import os
os.makedirs("/root/autodl-tmp/data/cifar224")

FileExistsError: [Errno 17] File exists: '/root/autodl-tmp/data/cifar224'

In [3]:
import json

with open("exps/Moal_cifar224_bilora.json", "r") as f:
    args = json.load(f)
print(type(args))

<class 'dict'>


In [4]:
from utils.data_manager import DataManager
from utils import factory
from utils.toolkit import count_parameters

In [5]:
data_manager = DataManager(
        args["dataset"],
        args["shuffle"],
        args["seed"],
        args["init_cls"],
        args["increment"],
        args,
        "/root/autodl-tmp/data/cifar224"
    )

[43, 92, 14, 57, 89, 20, 34, 69, 21, 3, 77, 33, 54, 86, 49, 79, 98, 15, 16, 61, 46, 59, 39, 12, 56, 50, 27, 87, 45, 78, 24, 0, 7, 51, 4, 73, 8, 35, 10, 13, 70, 74, 31, 55, 82, 5, 67, 75, 22, 76, 47, 83, 90, 1, 6, 84, 94, 66, 88, 91, 23, 29, 64, 37, 72, 18, 11, 44, 40, 93, 80, 30, 81, 68, 32, 58, 63, 2, 17, 65, 38, 41, 97, 48, 25, 28, 52, 99, 9, 36, 85, 62, 60, 96, 95, 53, 19, 42, 26, 71]


In [6]:
args["nb_classes"] = data_manager.nb_classes # update args
args["nb_tasks"] = data_manager.nb_tasks
args["model_name"] = "adapt_ac_com_sdc_ema_auto"

In [7]:
model = factory.get_model(args["model_name"], args)

This is for the BaseNet initialization.


/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).


After BaseNet initialization.


In [8]:
import logging
cnn_curve, nme_curve = {"top1": [], "top5": []}, {"top1": [], "top5": []}

all_his_acc = []
for task in range(data_manager.nb_tasks):
    logging.info("All params: {}".format(count_parameters(model._network)))
    logging.info(
        "Trainable params: {}".format(count_parameters(model._network, True))
    )
    model.incremental_train(data_manager)
    cnn_accy, nme_accy = model.eval_task()
    model.after_task()

    if nme_accy is not None:
        logging.info("CNN: {}".format(cnn_accy["grouped"]))
        logging.info("NME: {}".format(nme_accy["grouped"]))

        all_his_acc.append(cnn_accy["grouped"].values())
        cnn_curve["top1"].append(cnn_accy["top1"])
        cnn_curve["top5"].append(cnn_accy["top5"])

        nme_curve["top1"].append(nme_accy["top1"])
        nme_curve["top5"].append(nme_accy["top5"])

        logging.info("CNN top1 curve: {}".format(cnn_curve["top1"]))
        logging.info("CNN top5 curve: {}".format(cnn_curve["top5"]))
        logging.info("NME top1 curve: {}".format(nme_curve["top1"]))
        logging.info("NME top5 curve: {}\n".format(nme_curve["top5"]))

        print('Average Accuracy (CNN):', sum(cnn_curve["top1"])/len(cnn_curve["top1"]))
        print('Average Accuracy (NME):', sum(nme_curve["top1"])/len(nme_curve["top1"]))

        logging.info("Average Accuracy (CNN): {}".format(sum(cnn_curve["top1"])/len(cnn_curve["top1"])))
        logging.info("Average Accuracy (NME): {}".format(sum(nme_curve["top1"])/len(nme_curve["top1"])))
    else:
        logging.info("No NME accuracy.")
        logging.info("CNN: {}".format(cnn_accy["grouped"]))

        cnn_accy["grouped"].pop('total')
        cnn_accy["grouped"].pop('old')
        cnn_accy["grouped"].pop('new')
        all_his_acc.append(list(cnn_accy["grouped"].values()))

        cnn_curve["top1"].append(cnn_accy["top1"])
        cnn_curve["top5"].append(cnn_accy["top5"])

        logging.info("CNN top1 curve: {}".format(cnn_curve["top1"]))
        logging.info("CNN top5 curve: {}\n".format(cnn_curve["top5"]))

        print('Average Accuracy (CNN):', sum(cnn_curve["top1"])/len(cnn_curve["top1"]))
        logging.info("Average Accuracy (CNN): {} \n".format(sum(cnn_curve["top1"])/len(cnn_curve["top1"])))
logging.info("All History Accuracy (CNN): {} \n".format(all_his_acc))

Learning on 0-10
Train dataset size: 5000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


114,877,825 total parameters.
29,079,169 training parameters.
backbone.cls_token_grow 3840000
backbone.pos_embed_grow 918528
backbone.blocks.0.attn.lora_A_k.0.weight 49152
backbone.blocks.0.attn.lora_A_k.1.weight 49152
backbone.blocks.0.attn.lora_A_k.2.weight 49152
backbone.blocks.0.attn.lora_A_k.3.weight 49152
backbone.blocks.0.attn.lora_A_k.4.weight 49152
backbone.blocks.0.attn.lora_A_k.5.weight 49152
backbone.blocks.0.attn.lora_A_k.6.weight 49152
backbone.blocks.0.attn.lora_A_k.7.weight 49152
backbone.blocks.0.attn.lora_A_k.8.weight 49152
backbone.blocks.0.attn.lora_A_k.9.weight 49152
backbone.blocks.0.attn.lora_B_k.0.weight 49152
backbone.blocks.0.attn.lora_B_k.1.weight 49152
backbone.blocks.0.attn.lora_B_k.2.weight 49152
backbone.blocks.0.attn.lora_B_k.3.weight 49152
backbone.blocks.0.attn.lora_B_k.4.weight 49152
backbone.blocks.0.attn.lora_B_k.5.weight 49152
backbone.blocks.0.attn.lora_B_k.6.weight 49152
backbone.blocks.0.attn.lora_B_k.7.weight 49152
backbone.blocks.0.attn.lora_B

100%|██████████| 53/53 [02:19<00:00,  2.64s/it]


Task 0, Epoch 1/1 => Loss 2.286, Train_accy 20.56, Test_accy 36.80
This is for the BaseNet initialization.


...

100%|██████████| 6/6 [00:07<00:00,  1.28s/it]


Extracted vectors shape: (500, 768) float32
Extracted targets shape: (500,) int64
Class 9 covariance matrix shape: (768, 768)
Starting class alignment...


Alignment: 100%|██████████| 53/53 [01:10<00:00,  1.32s/batch]


Embedding shape:  torch.Size([5000, 5000])
Label shape torch.Size([5000])
One-hot label shape:  torch.Size([5000, 10])
Optimising ridge parameter...
selected lambda = 1000.0
gamma 1000.0
numpy inverse


AttributeError: `np.mat` was removed in the NumPy 2.0 release. Use `np.asmatrix` instead.

In [9]:
import numpy as np
print(np.__version__)

2.0.2


In [5]:
import torch

batch_size = 2 
dummy_input = torch.randn(
    batch_size,
    3, 224, 224,
    dtype=torch.float32,
    device="cpu"
)

print(dummy_input.shape)  # (B, 3, 224, 224)
print(dummy_input.dtype)  # torch.float32


torch.Size([2, 3, 224, 224])
torch.float32


In [6]:
y = model.forward(dummy_input, task=0)
y.shape

torch.Size([2, 768])

In [11]:
from Moal.backbone.vision_transformer_adapter import vit_base_patch16_224_adapter

In [12]:
from easydict import EasyDict
tuning_config = EasyDict(
    # AdaptFormer.
    ffn_adapt=True,
    ffn_option="parallel",
    ffn_adapter_layernorm_option="none",
    ffn_adapter_init_option="lora",
    ffn_adapter_scalar="0.1",
    ffn_num=64,
    d_model=768,
    # VPT related
    vpt_on=False,
    vpt_num=0,
)

In [13]:
ref_model = vit_base_patch16_224_adapter(num_classes=0,
                    global_pool=False, drop_path_rate=0.0, tuning_config=tuning_config)
ref_model.out_dim=768

I'm using ViT with adapters.


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

_IncompatibleKeys(missing_keys=['blocks.0.adaptmlp.down_proj.weight', 'blocks.0.adaptmlp.down_proj.bias', 'blocks.0.adaptmlp.up_proj.weight', 'blocks.0.adaptmlp.up_proj.bias', 'blocks.1.adaptmlp.down_proj.weight', 'blocks.1.adaptmlp.down_proj.bias', 'blocks.1.adaptmlp.up_proj.weight', 'blocks.1.adaptmlp.up_proj.bias', 'blocks.2.adaptmlp.down_proj.weight', 'blocks.2.adaptmlp.down_proj.bias', 'blocks.2.adaptmlp.up_proj.weight', 'blocks.2.adaptmlp.up_proj.bias', 'blocks.3.adaptmlp.down_proj.weight', 'blocks.3.adaptmlp.down_proj.bias', 'blocks.3.adaptmlp.up_proj.weight', 'blocks.3.adaptmlp.up_proj.bias', 'blocks.4.adaptmlp.down_proj.weight', 'blocks.4.adaptmlp.down_proj.bias', 'blocks.4.adaptmlp.up_proj.weight', 'blocks.4.adaptmlp.up_proj.bias', 'blocks.5.adaptmlp.down_proj.weight', 'blocks.5.adaptmlp.down_proj.bias', 'blocks.5.adaptmlp.up_proj.weight', 'blocks.5.adaptmlp.up_proj.bias', 'blocks.6.adaptmlp.down_proj.weight', 'blocks.6.adaptmlp.down_proj.bias', 'blocks.6.adaptmlp.up_proj.wei

In [15]:
ref_y = ref_model.forward(dummy_input)
ref_y.shape

torch.Size([2, 768])